In [3]:
a = 0
for i in range (6):
    a^=0
    a^=2
for i in range (16-6):
    a^=0

a^=0x0C
a

12

```
float pres_v;
  float pres_t;
  bool right_half;
  // Threshold behavior.
  calculate_torque_thresholds(torque_thresholds);
  counter = -1;
  for (auto DXL_ID : DXL_IDS) {
    counter++;
    if (counter > SERVOS - 1) break;
    else if (!mask[DXL_ID]) continue;

    if (dxl.getPresentPosition(DXL_ID, UNIT_DEGREE) < MIN_MAX_ANGLES.at(DXL_ID).first ||
        dxl.getPresentPosition(DXL_ID, UNIT_DEGREE) > MIN_MAX_ANGLES.at(DXL_ID).second) {
      dxl.ledOn(DXL_ID);
      set_operating_mode(DXL_ID, OP_CURRENT_BASED_POSITION);

      dxl.setGoalPosition(DXL_ID,
        clamp(
          dxl.getPresentPosition(DXL_ID, UNIT_DEGREE),
          MIN_MAX_ANGLES.at(DXL_ID).first,
          MIN_MAX_ANGLES.at(DXL_ID).second),
        UNIT_DEGREE);
      dxl.setGoalCurrent(DXL_ID, STATIC_CURRENT);
    } else {
      // Threshold behavior.
      pres_v = dxl.getPresentVelocity(DXL_ID);
      pres_t = dxl.getPresentCurrent(DXL_ID);
      if (threshold_crossed[DXL_ID]) {
        if (abs(pres_v) < 15) {
          set_operating_mode(DXL_ID, OP_CURRENT_BASED_POSITION);

          dxl.setGoalPosition(DXL_ID,
            //clamp(
                dxl.getPresentPosition(DXL_ID, UNIT_DEGREE),
            //  MIN_MAX_ANGLES.at(DXL_ID).first,
            //  MIN_MAX_ANGLES.at(DXL_ID).second),
            UNIT_DEGREE);
          dxl.setGoalCurrent(DXL_ID, STATIC_CURRENT);

          threshold_crossed[DXL_ID] = false;
        } else {
          dxl.ledOn(DXL_ID);
          dxl.setGoalCurrent(DXL_ID, MOVING_CURRENT * sgn(pres_v));
        }
      } else {
        dxl.ledOff(DXL_ID);
        right_half = in_quadrant_I_IV(DXL_ID);
        if ((!right_half && pres_t < torque_thresholds[counter] - 25) ||
            (right_half && pres_t > torque_thresholds[counter] + 25)) {
          dxl.ledOn(DXL_ID);
          dxl.setGoalPosition(DXL_ID,
            //clamp(
                dxl.getPresentPosition(DXL_ID, UNIT_DEGREE),
            //  MIN_MAX_ANGLES.at(DXL_ID).first,
            //  MIN_MAX_ANGLES.at(DXL_ID).second),
            UNIT_DEGREE);
          dxl.ledOff(DXL_ID);
        }
        if (abs(pres_v) > 20) {
          // Large adjustments.
          threshold_crossed[DXL_ID] = true;
          set_operating_mode(DXL_ID, OP_CURRENT);
        }
      }
    }
  }
```

```

// Safely set operating mode of motor.
void set_operating_mode(uint8_t DXL_ID, uint8_t mode) {
  // [t]orque_[e]nabled
  bool t_e = dxl.getTorqueEnableStat(DXL_ID);
  // Turn off torque when configuring items in EEPROM area.
  dxl.torqueOff(DXL_ID);
  dxl.setOperatingMode(DXL_ID, mode);
  // Reenable if torque was on prior.
  if (t_e) dxl.torqueOn(DXL_ID);
}

float get_level_angle(uint8_t DXL_ID) {
  // When arm is ready: fill in details.
  //case
  return dxl.getPresentPosition(DXL_ID, UNIT_DEGREE);
}

void get_level_angles() {
  counter = -1;
  for (auto DXL_ID : DXL_IDS) {
    counter++;
    level_angles[counter] = get_level_angle(DXL_ID);
  }
}

// Get angles relative to horizon.
void get_horizon_angles() {
  get_level_angles();
  for (int i = 0; i < SERVOS; i++) {
    if (i == 0) horizon_angles[i] = level_angles[i];
    else horizon_angles[i] = horizon_angles[i-1] + level_angles[i];
  }
}

// Calculate torque threshold constants. Docment later.
void calculate_torque_threshold_constants(float* torque_threshold_constants) {
  // Set startup angles, get leveled angles.
  for (auto DXL_ID : DXL_IDS) {
    dxl.ledOn(DXL_ID);
    dxl.torqueOn(DXL_ID);
    set_operating_mode(DXL_ID, OP_CURRENT_BASED_POSITION);
    dxl.setGoalCurrent(DXL_ID, STATIC_CURRENT);
    dxl.setGoalPosition(
      DXL_ID,
      clamp(STARTUP_ANGLES.at(DXL_ID),
            MIN_MAX_ANGLES.at(DXL_ID).first,
            MIN_MAX_ANGLES.at(DXL_ID).second),
      UNIT_DEGREE);
    delay(50);
    dxl.ledOff(DXL_ID);
    dxl.writeControlTableItem(ControlTableItem::PRESENT_POSITION, 0,1);
  }
  // Wait until motors are in position.
  delay(500);

  // Get angles relative to horizon.
  get_horizon_angles();

  // Calculate torque constants.
  float accum = 0.0;
  float m_t;
  counter = 6;
  for (auto rDXL_ID = DXL_IDS.rbegin(); rDXL_ID != DXL_IDS.rend(); ++rDXL_ID) {
    counter--;
    if (counter < 0) break;

    // [m]easured_[t]orque
    m_t = dxl.getPresentCurrent(*rDXL_ID);
    // Avoid division by 0.
    torque_threshold_constants[counter] = (m_t - accum)/(cos(level_angles[counter] * M_PI/180) || 0.001);
    accum += m_t;
  }
}

void calculate_torque_thresholds(float* torque_thresholds) {
  float accum = 0.0;
  float c_t;
  counter = 6;
  get_horizon_angles();
  for (auto rDXL_ID = DXL_IDS.rbegin(); rDXL_ID != DXL_IDS.rend(); ++rDXL_ID) {
    counter--;
    if (counter < 0) break;

    // [c]alculated_[t]orque
    c_t = torque_threshold_constants[counter]*cos(level_angles[counter] * M_PI/180);
    torque_thresholds[counter] = c_t + accum;
    accum += c_t;
  }
}


bool in_quadrant_I_IV(uint8_t DXL_ID) {
  float level_angle = get_level_angle(DXL_ID);
  return (level_angle < 90 || level_angle > 270);
}

```